In [406]:
pwd

'C:\\Users\\gherb\\Jupyter_Notebooks\\13_06_2022_Quranic_Studies'

In [1]:
from itertools import chain
from collections import Counter


In [2]:
import regex

In [3]:
import pandas as pd

In [5]:
import string_grouper

In [6]:
from string_grouper import compute_pairwise_similarities, group_similar_strings, match_most_similar, match_strings

In [7]:
def ngramize(x, n=3):
    x = x.split()
    return [
    ' '.join(x[i:i+n] )
    for i in range(
    len(x)-n+1
    )
        ]

In [8]:
with open('../DATASETS/Quran_Aya_Per_Line.txt', encoding='utf-8') as f:
    lines = [line.strip() for line in f.readlines()]

In [9]:
bigram = list(chain(*[ngramize(line, 2) for line in lines]))

In [10]:
trigram = list(chain(*[ngramize(line, 3) for line in lines]))

In [11]:
tetragram = list(chain(*[ngramize(line, 4) for line in lines]))

In [12]:
pentagram = list(chain(*[ngramize(line, 5) for line in lines]))

In [13]:
hexagram = list(chain(*[ngramize(line, 6) for line in lines]))

In [14]:
df_bigram = pd.DataFrame(data=bigram)

In [17]:
df_bigram = match_strings(df_bigram[0])

In [19]:
df_bigram = df_bigram[df_bigram.left_side != df_bigram.right_side].reset_index(drop=True)

In [135]:
def average_token_sim(x1, x2):
    try:
        x1 = x1.split()
        x2 = x2.split()
        container =  list(zip(x1, x2))
        container = [compute_pairwise_similarities(pd.Series(i[0]), pd.Series(i[1])) for i in container]
        container = pd.Series(container).mean()
        return container
    except:
        return (x1, x2)
        print(x1, x2)

In [136]:
test_01 = df_bigram.left_side.iloc[0]
test_02 = df_bigram.right_side.iloc[0]

In [137]:
test_03 = average_token_sim(test_01, test_02)

In [138]:
test_03

0.5

In [139]:
test_01, test_02

('له الرحمن', 'الله الرحمن')

In [142]:
df_bigram['L1'] = df_bigram.left_side.apply(lambda x: x.split()[0])
df_bigram['L2'] = df_bigram.left_side.apply(lambda x: x.split()[1])
df_bigram['R1'] = df_bigram.right_side.apply(lambda x: x.split()[0])
df_bigram['R2'] = df_bigram.right_side.apply(lambda x: x.split()[1])

In [145]:
df_bigram['L1_R1'] = compute_pairwise_similarities(df_bigram.L1, df_bigram.R1)
df_bigram['L2_R2'] = compute_pairwise_similarities(df_bigram.L2, df_bigram.R2)

In [148]:
df_bigram['MEAN_TOKEN_SIM'] = (df_bigram.L2_R2 + df_bigram.L1_R1) / 2

In [172]:
df_bigram = df_bigram[df_bigram['MEAN_TOKEN_SIM']>0.5].reset_index(drop=True)

In [237]:
df_bigram_beta = df_bigram.groupby(['L1'], as_index=False).agg({'R1': lambda x: x.tolist()})

In [238]:
df_bigram_beta = df_bigram_beta.reset_index(drop=True)

In [239]:
df_bigram_beta['LEN'] = df_bigram_beta['R1'].apply(lambda x: len(x))

In [240]:
df_bigram_beta = df_bigram_beta.sort_values(['LEN'], ascending=False).reset_index(drop=True)

In [241]:
df_bigram_beta.head(30)

,L1,R1,LEN
0,الذين,"[الذين, الذين, والذين, والذين, والذين, والذين,...",16249
1,والذين,"[الذين, الذين, الذين, الذين, الذين, الذين, الذ...",10581
2,الله,"[لله, الله, الله, بالله, بالله, الله, الله, ال...",8634
3,للذين,"[الذين, الذين, الذين, الذين, الذين, الذين, الذ...",6274
4,والله,"[الله, الله, الله, الله, الله, الله, الله, الل...",2721
5,وما,"[وما, وما, وما, وما, وما, وما, وما, وما, وما, ...",2447
6,على,"[على, على, على, على, وعلى, على, على, على, على,...",1347
7,فالذين,"[الذين, الذين, والذين, والذين, والذين, والذين,...",1202
8,بالله,"[الله, الله, الله, الله, الله, الله, الله, الل...",1171
9,لهم,"[ولهم, ولهم, ولهم, ولهم, ولهم, ولهم, ولهم, وله...",1101


In [242]:
df_bigram_beta['IDX_01'] = df_bigram_beta.index

In [243]:
df_bigram_beta['SET_R1'] = df_bigram_beta['R1'].apply(lambda x: set(x))

In [246]:
df_bigram_beta['DICT_TOKEN_IDX'] = df_bigram_beta.apply(lambda x: {i: x['IDX_01'] for i in x['SET_R1']}, axis=1)

In [250]:
df_bigram_beta.sample(3)

,L1,R1,LEN,IDX_01,SET_R1,DICT_TOKEN_IDX
178,كفى,"[وكفى, وكفى, وكفى, وكفى, وكفى, وكفى, وكفى, وكف...",42,178,"{فكفى, وكفى}","{'فكفى': 178, 'وكفى': 178}"
124,جاء,"[جاء, جاء, جاء, جاء, جاء, جاء, جاء, وجاء, وجاء...",67,124,"{وجاء, جاء, فجاء}","{'وجاء': 124, 'جاء': 124, 'فجاء': 124}"
245,ومما,"[مما, مما, مما, مما, مما, مما, مما, مما, مما, ...",25,245,{مما},{'مما': 245}


In [253]:
df_bigram_beta[df_bigram_beta['L1'] == 'مما']

,L1,R1,LEN,IDX_01,SET_R1,DICT_TOKEN_IDX
139,مما,"[ومما, ومما, ومما, ومما, مما, مما, مما, مما, م...",59,139,"{ومما, وما, مما}","{'ومما': 139, 'وما': 139, 'مما': 139}"


In [371]:
dict_container = dict()

In [372]:
for (idx) in range(len(df_bigram_beta)):
    token = df_bigram_beta['L1'].iloc[idx]
    inner_set = df_bigram_beta['SET_R1'].iloc[idx]
    
    if token not in dict_container:
        dict_container[token] = (idx, token)
        
    for token_inner in inner_set:
        if token_inner not in dict_container:
            try:
                pass_01 = dict_container[token]
                dict_container[token_inner] = pass_01
            except:
                dict_container[token_inner] = (idx, token)

In [377]:
dict_container['ينظروا']

(490, 'فانظروا')

In [378]:
df_test = pd.DataFrame(dict_container.items())

In [384]:
df_test[2] = df_test[1].apply(lambda x: x[1])

In [423]:
df_test[df_test[0].apply(lambda x: 'لله' in x)]

,0,1,2
6,الله,"(2, الله)",الله
7,فالله,"(2, الله)",الله
9,ولله,"(2, الله)",الله
10,والله,"(2, الله)",الله
12,بالله,"(2, الله)",الله
13,أبالله,"(2, الله)",الله
14,تالله,"(2, الله)",الله
16,لله,"(2, الله)",الله
42,فلله,"(2, الله)",الله


In [400]:
x = 'word'

In [401]:
x[0]

'w'

In [402]:
x[-1]

'd'

In [404]:
x[0+1:-1]

'or'

In [424]:
dict_bigram_normalize = dict(zip(df_test[0], df_test[2]))

In [425]:
df_bigram['L1_NORM'] = df_bigram['L1'].apply(lambda x: dict_bigram_normalize.get(x, x))

In [426]:
df_bigram['L2_NORM'] = df_bigram['L2'].apply(lambda x: dict_bigram_normalize.get(x, x))

In [427]:
df_bigram

,left_index,left_side,similarity,right_side,right_index,L1,L2,R1,R2,L1_R1,L2_R2,MEAN_TOKEN_SIM,L1_NORM,L2_NORM
0,16143,والحمد لله,0.972541,الحمد لله,3,والحمد,لله,الحمد,لله,0.966295,1.000000,0.983147,والحمد,الله
1,54584,والحمد لله,0.972541,الحمد لله,3,والحمد,لله,الحمد,لله,0.966295,1.000000,0.983147,والحمد,الله
2,68410,بيوم الدين,0.890154,يوم الدين,8,بيوم,الدين,يوم,الدين,0.563122,1.000000,0.781561,يوم,للدين
3,69304,بيوم الدين,0.890154,يوم الدين,8,بيوم,الدين,يوم,الدين,0.563122,1.000000,0.781561,يوم,للدين
4,70338,بيوم الدين,0.890154,يوم الدين,8,بيوم,الدين,يوم,الدين,0.563122,1.000000,0.781561,يوم,للدين
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93253,57432,أغنى عنهم,0.969627,أغنى عنه,71505,أغنى,عنهم,أغنى,عنه,1.000000,0.803512,0.901756,أغنى,لهم
93254,60944,أغنى عنهم,0.969627,أغنى عنه,71505,أغنى,عنهم,أغنى,عنه,1.000000,0.803512,0.901756,أغنى,لهم
93255,68766,الله أحدا,0.876161,الله أحد,71520,الله,أحدا,الله,أحد,1.000000,0.679115,0.839558,الله,أحدا
93256,68864,الله أحدا,0.876161,الله أحد,71520,الله,أحدا,الله,أحد,1.000000,0.679115,0.839558,الله,أحدا


In [443]:
df_bigram['BIAGRAM_NORM'] = df_bigram.apply(lambda x: f"""{x['L1_NORM']} {x["L2_NORM"]}""", axis=1)

In [444]:
dict_top_200_biagrams_normalized = dict(df_bigram['BIAGRAM_NORM'].value_counts())

In [445]:
df_top_200_biagrams_normalized = pd.DataFrame(dict_top_200_biagrams_normalized.items())

In [448]:
df_top_200_biagrams_normalized[df_top_200_biagrams_normalized[1]>=42]

,0,1
0,الذين آمنوا,22962
1,الذين كفروا,10318
2,على الله,2376
3,واتقوا الله,1788
4,الله الذي,1756
...,...,...
198,الله وهو,42
199,ذكر الله,42
200,وما نصيرا,42
201,وما تتبع,42


In [450]:
#df_top_200_biagrams_normalized.to_pickle(F'{"df_top_200_biagrams_normalized".upper()}.pkl', protocol=4)

In [457]:
#df_bigram.to_pickle('df_bigram.pkl', protocol=4)

In [458]:
#df_bigram_beta.to_pickle('df_bigram_beta.pkl', protocol=4)

In [459]:
#df_test.to_pickle('NORMALIZED_WORDS_DF.pkl', protocol=4)

In [460]:
df_bigram

,left_index,left_side,similarity,right_side,right_index,L1,L2,R1,R2,L1_R1,L2_R2,MEAN_TOKEN_SIM,L1_NORM,L2_NORM,BIAGRAM_NORM
0,16143,والحمد لله,0.972541,الحمد لله,3,والحمد,لله,الحمد,لله,0.966295,1.000000,0.983147,والحمد,الله,والحمد الله
1,54584,والحمد لله,0.972541,الحمد لله,3,والحمد,لله,الحمد,لله,0.966295,1.000000,0.983147,والحمد,الله,والحمد الله
2,68410,بيوم الدين,0.890154,يوم الدين,8,بيوم,الدين,يوم,الدين,0.563122,1.000000,0.781561,يوم,للدين,يوم للدين
3,69304,بيوم الدين,0.890154,يوم الدين,8,بيوم,الدين,يوم,الدين,0.563122,1.000000,0.781561,يوم,للدين,يوم للدين
4,70338,بيوم الدين,0.890154,يوم الدين,8,بيوم,الدين,يوم,الدين,0.563122,1.000000,0.781561,يوم,للدين,يوم للدين
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93253,57432,أغنى عنهم,0.969627,أغنى عنه,71505,أغنى,عنهم,أغنى,عنه,1.000000,0.803512,0.901756,أغنى,لهم,أغنى لهم
93254,60944,أغنى عنهم,0.969627,أغنى عنه,71505,أغنى,عنهم,أغنى,عنه,1.000000,0.803512,0.901756,أغنى,لهم,أغنى لهم
93255,68766,الله أحدا,0.876161,الله أحد,71520,الله,أحدا,الله,أحد,1.000000,0.679115,0.839558,الله,أحدا,الله أحدا
93256,68864,الله أحدا,0.876161,الله أحد,71520,الله,أحدا,الله,أحد,1.000000,0.679115,0.839558,الله,أحدا,الله أحدا


In [464]:
df_top_200_biagrams_normalized[0]

,0,1
0,الذين آمنوا,22962
1,الذين كفروا,10318
2,على الله,2376
3,واتقوا الله,1788
4,الله الذي,1756
...,...,...
2960,البينات الذي,1
2961,يرسل عليهم,1
2962,ولو جئتكم,1
2963,أرضكم بسحرهما,1
